In [63]:
import requests
from bs4 import BeautifulSoup
from flask import Flask, jsonify
import schedule
import time

url_base = 'https://simple.ripley.cl/deporte-y-aventura/zapatillas/todo-zapatillas?facet%5B0%5D=G%C3%A9nero%3AHombre&facet%5B1%5D=Marca%3AADIDAS&facet%5B2%5D=Marca%3ANIKE&facet%5B3%5D=Marca%3APUMA&facet%5B4%5D=Talla%3A39.5&facet%5B5%5D=Talla%3A40&facet%5B6%5D=Talla%3A40.5&facet%5B7%5D=Talla%3A41&facet%5B8%5D=Talla%3A41.5&orderBy=price_asc&s=mdco&source=search&term=apatillas&page='

agent = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}

def clean_price(price):
    return int(price.replace('$', '').replace('.', ''))

def get_clean_price(product, title):
    try:
        price = clean_price(product.find('li', title=title).text)
    except:
        price = -1
    return price

In [70]:
data = []
for pagina in range(1, 6):
    page = requests.get(url_base+str(pagina), headers=agent)
    soup = BeautifulSoup(page.content, 'html.parser')
    products = soup.find_all('div', class_='catalog-product-details')
    for product in products:
        # Get name
        name = product.find('div', class_='catalog-product-details__name').text
        # Get prices
        normal_price = get_clean_price(product, 'Precio Normal')
        internet_price = get_clean_price(product, 'Precio Internet')
        ripley_price = get_clean_price(product, 'Precio Ripley')
        color = product.find('div', class_='catalog-colors-option-outer').get('title')
        data.append({'name': name, 'normal_price': normal_price, 'internet_price': internet_price, 'ripley_price': ripley_price, 'color': color})


<Response [200]>
1 [<div class="catalog-product-details"><div class="catalog-product-emblems__desktop"></div><div class="catalog-product-emblems__mobile"></div><div class="catalog-product-name-container"><div class="catalog-product-details__logo-container"><div class="brand-logo"><span>PUMA</span></div></div></div><div class="catalog-product-details__name">SANDALIAS PUMA BMW MMS LOGO LEADCAT 2.0</div><div aria-hidden="true" class="product-rating product-rating-small" role="button" tabindex="0"><div class="stars-ranking stars-ranking-sprite--small stars-ranking-0"></div><span class="product-rating-text">(<span>Sin</span> <!-- -->Calificaciones)</span></div><div class="catalog-product-details__prices"><div class="catalog-prices"><ul class="catalog-prices__list"><li class="catalog-prices__list-price catalog-prices__lowest catalog-prices__line_thru" title="Precio Normal">$29.990</li><li class="catalog-prices__offer-price" title="Precio Internet">$20.990</li><li class="catalog-prices__card-

In [71]:
len(data)
# data

130

In [ ]:
# Get the data
data = []
for i in range(2, 5):
    url = url_base + str(i)
    page = requests.get(url, headers=agent)
    soup = BeautifulSoup(page.content, 'html.parser')
    for product in soup.find_all('div', class_='catalog-product-details'):
        try:
            name = product.find('div', class_='catalog-product-details__name').text
            before_price = product.find('div', class_='catalog-prices__lowest').text
            # select by title
            normal_price = product.find('div', title='Precio Internet').text
            ripley_price = product.find('div', title='Precio Ripley').text

            data.append({'name': name, 'before_price': before_price, 'normal_price': normal_price, 'ripley_price': ripley_price})
        except:
            print("Error en producto " + str(i) + "")
print(data)

In [1]:
import requests
from bs4 import BeautifulSoup
from flask import Flask, jsonify
import schedule
import time

app = Flask(__name__)

url_base = 'https://simple.ripley.cl/deporte-y-aventura/zapatillas/todo-zapatillas?facet%5B0%5D=G%C3%A9nero%3AHombre&facet%5B1%5D=Marca%3AADIDAS&facet%5B2%5D=Marca%3ANIKE&facet%5B3%5D=Marca%3APUMA&facet%5B4%5D=Talla%3A39.5&facet%5B5%5D=Talla%3A40&facet%5B6%5D=Talla%3A40.5&facet%5B7%5D=Talla%3A41&facet%5B8%5D=Talla%3A41.5&orderBy=price_asc&s=mdco&source=search&term=apatillas&page='
def scrape():
    # Get the web page to scrape
    data = []
    try:
        page = requests.get(url_base)
        soup = BeautifulSoup(page.content, 'html.parser')

        # Get the data
        for i in range(1, 5):
            url = url_base + str(i)
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')
            for product in soup.find_all('div', class_='catalog-product-details'):
                name = product.find('div', class_='catalog-product-details__name').text
                before_price = product.find('div', class_='catalog-prices__lowest').text
                # select by title
                normal_price = product.find('div', title='Precio Internet').text
                ripley_price = product.find('div', title='Precio Ripley').text

                data.append({'name': name, 'before_price': before_price, 'normal_price': normal_price, 'ripley_price': ripley_price})

        
    except:
        print("Error")
    

    # Return the data as a JSON response
    print(data)
    return jsonify(data)

@app.route('/crawl')
def crawl():
    return scrape()

# Schedule the scraping task to run every 30 minutes
schedule.every(30).minutes.do(scrape)

if __name__ == '__main__':
    # Start the Flask app
    app.run()

    # Start the scheduling loop
    while True:
        schedule.run_pending()
        time.sleep(1)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
